<a href="https://colab.research.google.com/github/andresgiraldo3312/DMF/blob/main/DMF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import pandas as pd
from numpy.linalg import inv


class DenseNet(torch.nn.Module):
    def __init__(self, n_sensores, n_tiempo, n_factors):
        """
        Simple Feedforward with Embeddings
        """
        super().__init__()
       
       	# sensores and tiempdo embedding layers
        self.sensores_factors = torch.nn.Embedding(n_sensores, n_factors)
        self.tiempo_factors = torch.nn.Embedding(n_tiempo, n_factors)

   	    # linear layers
        self.linear1 = torch.nn.Linear(n_factors * 2, 1024, bias = True)
        self.linear2 = torch.nn.Linear(1024, 512, bias = True)
        self.linear3 = torch.nn.Linear(512, 128, bias = True)
        self.linear4 = torch.nn.Linear(128, 32, bias = True)
        self.linear5 = torch.nn.Linear(32, 1, bias = True)




    def forward(self, sensores, tiempo):
        sensores_embedding = self.sensores_factors(sensores)
        tiempo_embedding = self.tiempo_factors(tiempo) 
	      
        # concatenate sensores and tiempo embeddings to form input
        x_in = torch.cat([sensores_embedding, tiempo_embedding], 1)

        x = torch.nn.functional.relu(self.linear1(x_in))
        x = torch.nn.functional.dropout(x,p=0.5, training=self.training)
        x = torch.nn.functional.relu(self.linear2(x))
        x = torch.nn.functional.relu(self.linear3(x))
        x = torch.nn.functional.relu(self.linear4(x))

        x = self.linear5(x)
        return torch.sigmoid(x)

    def predict(self, sensores, tiempo):
        # return the score
        output_scores = self.forward(sensores, tiempo)
        return output_scores



def DMF(db, n_factors, Epoca, batch_size, device):
    
    Red_Sensores = np.asarray(db) 
    
    maximo = np.amax(Red_Sensores)
    
    Y1 = np.asarray(Red_Sensores)/maximo

    rows, cols = np.nonzero(Y1)                                                                    # Solo se trabaja con los datos diferentes de cero                                                                
    X = np.zeros((len(rows),3))                                                                    # coordenada sensor vs tiempo
    
    X[:,0] = rows
    X[:,1] = cols
    X[:,2] = Y1[rows,cols]
    
    X = np.random.permutation(X)                                                                   # Baraja las filas (reorganiza los datos)
    N_train = int(0.9*len(rows))                                                                   # Porcentaje de datos de entrenamiento y validaciòn
    train = X[0:N_train,:]                                                                         # Entrenamiento
    valid = X[N_train:-1,:]    
    
    n_sensores,n_tiempo = Red_Sensores.shape
    
    train_index = torch.LongTensor(train[:,0:2])
    train_ratings = torch.Tensor(train[:,2])
    train_tensor = torch.utils.data.TensorDataset(train_index, train_ratings) 
    train_loader = torch.utils.data.DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

    valid_index = torch.LongTensor(valid[:,0:2])
    valid_ratings = torch.Tensor(valid[:,2])
    valid_tensor = torch.utils.data.TensorDataset(valid_index, valid_ratings) 
    valid_loader = torch.utils.data.DataLoader(valid_tensor, batch_size=batch_size, shuffle=True)
    
    model = DenseNet(n_sensores, n_tiempo, n_factors)
    model.to(device)

    loss_fn = torch.nn.MSELoss(reduction = 'mean')
    optimizer = torch.optim.Adam(model.parameters())

    loss_mse = []

    for t in range(Epoca):
        n = 0
        m = 0
        loss_epoca = 0

        model.train(True)
        
        for batch,dat in enumerate(train_loader):
            optimizer.zero_grad()
        
            # get sensor, tiempo and rating data
            sensor = dat[0][:,0]
            tiempo = dat[0][:,1]
            rating = dat[1].reshape(-1,1)
        
            # predict
            prediction = model(sensor.to(device), tiempo.to(device))
            loss = loss_fn(prediction, rating.to(device))
            loss_epoca = loss_epoca + loss.item()
            n = n+1

            # backpropagate
            loss.backward()

            # update weights
            optimizer.step()
        
        loss_mse.append(loss_epoca/n)
        print(f'epoch : [{t+1}/{Epoca}], loss_train: {loss_epoca/n:.6f}') 

    matriz_estimada_ = np.zeros((n_sensores,n_tiempo)) 
    
    for i in range(n_sensores):
        for j in range(n_tiempo):
            sensor = Variable(torch.LongTensor([i]))
            tiempo = Variable(torch.LongTensor([j]))
            matriz_estimada_[i,j] = model(sensor.to(device), tiempo.to(device))

    matriz_estimada = matriz_estimada_*maximo

    data_estimada = pd.DataFrame(matriz_estimada)  
    
    return data_estimada


In [ ]:
import torch
import numpy as np
import pandas as pd
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for mes in range(6):
  print("=============== MES " + str(mes+1) + " ===============")
  db = pd.read_csv('DatosEliminados/Ventana_Eli_mes' + str(mes+1) + '.csv', sep = ',', index_col = 0)
  estimados = DMF(db, n_factors = 512, Epoca = 100, batch_size = 64, device = device)
  estimados.to_csv("DatosEstimados/DMF_1/Ventana_Estimada_mes" + str(mes+1) + ".csv")

